In [1]:
from misophonia_dataset._log import setup_print_logging

setup_print_logging()

import pandas as pd

from misophonia_dataset._analysis import models_to_df
from misophonia_dataset.source_data.esc50 import Esc50Dataset
from misophonia_dataset.source_data.fsd50k import Fsd50kDataset
from misophonia_dataset.source_data.foams import FoamsDataset

# Loading individual source data

In [2]:
esc50 = Esc50Dataset()
esc50.download_data()
esc50_meta = models_to_df(esc50.get_metadata())

esc50_meta.head()

,split,source_dataset,file_path,freesound_id,label_type,labels,validated_by,sound_license,dataset_license,esc50_filename,esc50_fold,esc50_target,esc50_category,esc50_esc10,esc50_take,_model
0,train,ESC50,data/ESC50/ESC-50-master/audio/1-12653-A-15.wav,12653,trigger,"(water_drops,)",None,{'license_url': 'http://creativecommons.org/li...,{'license_url': 'https://creativecommons.org/l...,1-12653-A-15.wav,1,15,water_drops,False,A,split='train' source_dataset='ESC50' file_path...
1,train,ESC50,data/ESC50/ESC-50-master/audio/1-12654-A-15.wav,12654,trigger,"(water_drops,)",None,{'license_url': 'http://creativecommons.org/li...,{'license_url': 'https://creativecommons.org/l...,1-12654-A-15.wav,1,15,water_drops,False,A,split='train' source_dataset='ESC50' file_path...
2,train,ESC50,data/ESC50/ESC-50-master/audio/1-12654-B-15.wav,12654,trigger,"(water_drops,)",None,{'license_url': 'http://creativecommons.org/li...,{'license_url': 'https://creativecommons.org/l...,1-12654-B-15.wav,1,15,water_drops,False,B,split='train' source_dataset='ESC50' file_path...
3,train,ESC50,data/ESC50/ESC-50-master/audio/1-137-A-32.wav,137,trigger,"(typing,)",None,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'https://creativecommons.org/l...,1-137-A-32.wav,1,32,keyboard_typing,False,A,split='train' source_dataset='ESC50' file_path...
4,val,ESC50,data/ESC50/ESC-50-master/audio/1-16746-A-15.wav,16746,trigger,"(water_drops,)",None,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'https://creativecommons.org/l...,1-16746-A-15.wav,1,15,water_drops,False,A,split='val' source_dataset='ESC50' file_path=P...


In [3]:
fsd50k = Fsd50kDataset()
fsd50k.download_data()
fsd50k_meta = models_to_df(fsd50k.get_metadata())

assert fsd50k_meta.loc[fsd50k_meta["fsd50k_split"] == "eval", "split"].isin(["test"]).all(), (
    "All samples from FSD50K 'eval' must be in the 'test' split"
)

fsd50k_meta.head()

,split,source_dataset,file_path,freesound_id,label_type,labels,validated_by,sound_license,dataset_license,fsd50k_fname,fsd50k_labels,fsd50k_mids,fsd50k_split,_model
0,train,FSD50K,data/FSD50K/dev_audio/64760.wav,64760,background,"(Electric_guitar,)",None,{'license_url': 'http://creativecommons.org/li...,{'license_url': 'https://creativecommons.org/l...,64760,[Electric_guitar],/m/02sgy,dev,split='train' source_dataset='FSD50K' file_pat...
1,train,FSD50K,data/FSD50K/dev_audio/16399.wav,16399,background,"(Electric_guitar,)",None,{'license_url': 'http://creativecommons.org/li...,{'license_url': 'https://creativecommons.org/l...,16399,[Electric_guitar],/m/02sgy,dev,split='train' source_dataset='FSD50K' file_pat...
2,train,FSD50K,data/FSD50K/dev_audio/16401.wav,16401,background,"(Electric_guitar,)",None,{'license_url': 'http://creativecommons.org/li...,{'license_url': 'https://creativecommons.org/l...,16401,[Electric_guitar],/m/02sgy,dev,split='train' source_dataset='FSD50K' file_pat...
3,train,FSD50K,data/FSD50K/dev_audio/16402.wav,16402,background,"(Electric_guitar,)",None,{'license_url': 'http://creativecommons.org/li...,{'license_url': 'https://creativecommons.org/l...,16402,[Electric_guitar],/m/02sgy,dev,split='train' source_dataset='FSD50K' file_pat...
4,train,FSD50K,data/FSD50K/dev_audio/16404.wav,16404,background,"(Electric_guitar,)",None,{'license_url': 'http://creativecommons.org/li...,{'license_url': 'https://creativecommons.org/l...,16404,[Electric_guitar],/m/02sgy,dev,split='train' source_dataset='FSD50K' file_pat...


In [4]:
foams = FoamsDataset()
foams.download_data()
foams_meta = models_to_df(foams.get_metadata())

assert foams_meta["split"].nunique() == 1 and foams_meta["split"].unique()[0] == "test", (
    "All FOAMS samples must be in the 'test' split"
)

foams_meta.head()

,split,source_dataset,file_path,freesound_id,label_type,labels,validated_by,sound_license,dataset_license,foams_duration,_model
0,test,FOAMS,data/FOAMS/processed_audio/184320_processed.wav,184320,trigger,"(basketball_dribbling,)","(FOAMS,)",{'license_url': 'http://creativecommons.org/pu...,{'license_url': 'https://creativecommons.org/l...,3.270003,split='test' source_dataset='FOAMS' file_path=...
1,test,FOAMS,data/FOAMS/processed_audio/580220_processed.wav,580220,trigger,"(basketball_dribbling,)","(FOAMS,)",{'license_url': 'http://creativecommons.org/pu...,{'license_url': 'https://creativecommons.org/l...,3.862957,split='test' source_dataset='FOAMS' file_path=...
2,test,FOAMS,data/FOAMS/processed_audio/262507_processed.wav,262507,trigger,"(basketball_dribbling,)","(FOAMS,)",{'license_url': 'http://creativecommons.org/pu...,{'license_url': 'https://creativecommons.org/l...,4.903607,split='test' source_dataset='FOAMS' file_path=...
3,test,FOAMS,data/FOAMS/processed_audio/453757_processed.wav,453757,trigger,"(basketball_dribbling,)","(FOAMS,)",{'license_url': 'http://creativecommons.org/pu...,{'license_url': 'https://creativecommons.org/l...,5.097981,split='test' source_dataset='FOAMS' file_path=...
4,test,FOAMS,data/FOAMS/processed_audio/530362_processed.wav,530362,trigger,"(basketball_dribbling,)","(FOAMS,)",{'license_url': 'http://creativecommons.org/pu...,{'license_url': 'https://creativecommons.org/l...,5.344224,split='test' source_dataset='FOAMS' file_path=...


# Combining all source data

In [5]:
combined_meta = pd.concat([esc50_meta, fsd50k_meta, foams_meta], ignore_index=True)
combined_meta = combined_meta[
    # These colums are there, see the interface.SourceMetaData class
    ["source_dataset", "split", "freesound_id", "label_type", "labels", "validated_by", "file_path", "dataset_license", "sound_license"] 
]

## Examine different subsets

In [6]:
from IPython.display import HTML, display

def print_value_counts_pcts(title: str, col: pd.Series) -> None:
    normalized = col.value_counts(normalize=True, dropna=False)
    counts = col.value_counts(dropna=False)
    # if all the same, just print that
    if len(counts) == 1:
        print(f"{title} All samples have the same value: {counts.index[0]}")
        return
    
    dist = pd.DataFrame({"count": counts, "percentage": normalized * 100}, index=counts.index)
    print(title)
    display(dist)

def print_subset_info(subset_descr: str, subset: pd.DataFrame):
    display(HTML(f"<h3>=== Subset: {subset_descr} ===</h3>"))
    print(f"* Total samples: {len(subset)}")
    print_value_counts_pcts("\n* Distribution by source dataset:", subset["source_dataset"])
    print_value_counts_pcts("\n* Distribution by split:", subset["split"])
    print_value_counts_pcts("\n* Distribution by label type:", subset["label_type"])
    print_value_counts_pcts("\n* Distribution by validated by:", subset["validated_by"])
    print("\n* Sample entries:")
    display(subset.sample(min(3, len(subset))))

### Examine all source data

In [7]:
print_subset_info("All source data", combined_meta)

* Total samples: 31241

* Distribution by source dataset:


,count,percentage
source_dataset,,
FSD50K,30991,99.199770
ESC50,200,0.640184
FOAMS,50,0.160046



* Distribution by split:


,count,percentage
split,,
train,23474,75.138440
test,4586,14.679428
val,3181,10.182132



* Distribution by label type:


,count,percentage
label_type,,
background,27930,89.401748
trigger,2417,7.736628
control,894,2.861624



* Distribution by validated by:


,count,percentage
validated_by,,
None,31179,99.801543
"(FOAMS,)",62,0.198457



* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,dataset_license,sound_license
17894,FSD50K,train,60442,background,"(Splash_and_splatter,)",None,data/FSD50K/dev_audio/60442.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...
24118,FSD50K,train,27959,background,"(Sink_(filling_or_washing),)",None,data/FSD50K/dev_audio/27959.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
3856,FSD50K,train,66751,background,"(Wind,)",None,data/FSD50K/dev_audio/66751.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...


### Examine by label type

In [8]:
# For each label type
for label_type in combined_meta["label_type"].unique():
    subset = combined_meta[combined_meta["label_type"] == label_type]
    print_subset_info(f"label_type = {label_type}", subset)

* Total samples: 2417

* Distribution by source dataset:


,count,percentage
source_dataset,,
FSD50K,2167,89.656599
ESC50,200,8.274721
FOAMS,50,2.068680



* Distribution by split:


,count,percentage
split,,
train,1646,68.100952
test,548,22.672735
val,223,9.226314



* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by:


...

,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,dataset_license,sound_license
3039,FSD50K,train,335946,background,"(Writing,)",None,data/FSD50K/dev_audio/335946.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
5949,FSD50K,val,354658,background,"(Flute,)",None,data/FSD50K/dev_audio/354658.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...
2017,FSD50K,train,93872,background,"(Male_speech_and_man_speaking,)",None,data/FSD50K/dev_audio/93872.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...


* Total samples: 894

* Distribution by source dataset: All samples have the same value: FSD50K

* Distribution by split:


,count,percentage
split,,
train,675,75.503356
test,128,14.317673
val,91,10.178971



* Distribution by label type: All samples have the same value: control

* Distribution by validated by: All samples have the same value: None

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,dataset_license,sound_license
30437,FSD50K,test,85578,control,"(harp,)",None,data/FSD50K/eval_audio/85578.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...
22605,FSD50K,train,4252,control,"(harp,)",None,data/FSD50K/dev_audio/4252.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...
28971,FSD50K,test,56921,control,"(piano,)",None,data/FSD50K/eval_audio/56921.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...


### Examine by trigger category

In [9]:
# For each trigger sound label
for label in combined_meta[combined_meta["label_type"] == "trigger"]["labels"].explode().unique():
    subset = combined_meta[
        (combined_meta["label_type"] == "trigger") & (combined_meta["labels"].apply(lambda labels: label in labels))
    ]
    print_subset_info(f"Trigger = {label}", subset)

* Total samples: 328

* Distribution by source dataset:


,count,percentage
source_dataset,,
FSD50K,283,86.280488
ESC50,40,12.195122
FOAMS,5,1.524390



* Distribution by split:


,count,percentage
split,,
train,230,70.121951
test,63,19.207317
val,35,10.670732



* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by:


...

,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,dataset_license,sound_license
22179,FSD50K,train,252796,trigger,"(knife_cutting,)",None,data/FSD50K/dev_audio/252796.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...
26508,FSD50K,train,125521,trigger,"(knife_cutting,)",None,data/FSD50K/dev_audio/125521.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
26506,FSD50K,val,369233,trigger,"(knife_cutting,)",None,data/FSD50K/dev_audio/369233.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...


* Total samples: 5

* Distribution by source dataset: All samples have the same value: FOAMS

* Distribution by split: All samples have the same value: test

* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by: All samples have the same value: ('FOAMS',)

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,dataset_license,sound_license
31195,FOAMS,test,530362,trigger,"(basketball_dribbling,)","(FOAMS,)",data/FOAMS/processed_audio/530362_processed.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
31194,FOAMS,test,453757,trigger,"(basketball_dribbling,)","(FOAMS,)",data/FOAMS/processed_audio/453757_processed.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
31191,FOAMS,test,184320,trigger,"(basketball_dribbling,)","(FOAMS,)",data/FOAMS/processed_audio/184320_processed.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...


* Total samples: 5

* Distribution by source dataset: All samples have the same value: FOAMS

* Distribution by split: All samples have the same value: test

* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by: All samples have the same value: ('FOAMS',)

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,dataset_license,sound_license
31210,FOAMS,test,275304,trigger,"(flipping_newspaper,)","(FOAMS,)",data/FOAMS/processed_audio/275304_processed.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
31206,FOAMS,test,636467,trigger,"(flipping_newspaper,)","(FOAMS,)",data/FOAMS/processed_audio/636467_processed.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
31207,FOAMS,test,473981,trigger,"(flipping_newspaper,)","(FOAMS,)",data/FOAMS/processed_audio/473981_processed.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...


### Examine by source dataset

In [10]:
# for each source dataset
for source_dataset in combined_meta["source_dataset"].unique():
    subset = combined_meta[combined_meta["source_dataset"] == source_dataset]
    print_subset_info(f"Source dataset = {source_dataset}", subset)

* Total samples: 200

* Distribution by source dataset: All samples have the same value: ESC50

* Distribution by split:


,count,percentage
split,,
train,131,65.5
test,53,26.5
val,16,8.0



* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by:


,count,percentage
validated_by,,
None,194,97.0
"(FOAMS,)",6,3.0



* Sample entries:


...

,count,percentage
validated_by,,
None,30985,99.98064
"(FOAMS,)",6,0.01936



* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,dataset_license,sound_license
8748,FSD50K,train,58672,background,"(Crowd, Applause, Cheering)",None,data/FSD50K/dev_audio/58672.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...
5324,FSD50K,train,54404,background,"(Train, Engine)",None,data/FSD50K/dev_audio/54404.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
11567,FSD50K,train,144780,background,"(Snare_drum,)",None,data/FSD50K/dev_audio/144780.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...


* Total samples: 50

* Distribution by source dataset: All samples have the same value: FOAMS

* Distribution by split: All samples have the same value: test

* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by: All samples have the same value: ('FOAMS',)

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,dataset_license,sound_license
31191,FOAMS,test,184320,trigger,"(basketball_dribbling,)","(FOAMS,)",data/FOAMS/processed_audio/184320_processed.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
31210,FOAMS,test,275304,trigger,"(flipping_newspaper,)","(FOAMS,)",data/FOAMS/processed_audio/275304_processed.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
31227,FOAMS,test,267932,trigger,"(swallowing,)","(FOAMS,)",data/FOAMS/processed_audio/267932_processed.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...


### Examine single vs. multi-label data

In [11]:
def normalize_label_tuple(s: pd.Series) -> pd.Series:
    return s.apply(lambda labels: tuple(sorted(labels)))

label_len = combined_meta["labels"].apply(len)
for label_count in sorted(label_len.unique()):
    subset = combined_meta[label_len == label_count]
    print_subset_info(f"Number of labels = {label_count}", subset)
    print_value_counts_pcts("\n* Label pairs distribution:", normalize_label_tuple(subset["labels"]))

    print("\n ---- ")

* Total samples: 28117

* Distribution by source dataset:


,count,percentage
source_dataset,,
FSD50K,27867,99.110858
ESC50,200,0.711313
FOAMS,50,0.177828



* Distribution by split:


,count,percentage
split,,
train,21666,77.056585
test,3528,12.547569
val,2923,10.395846



* Distribution by label type:


...

* Total samples: 2

* Distribution by source dataset: All samples have the same value: FSD50K

* Distribution by split: All samples have the same value: test

* Distribution by label type: All samples have the same value: background

* Distribution by validated by: All samples have the same value: None

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,dataset_license,sound_license
27641,FSD50K,test,368610,background,"(Synthesizer, Snare_drum, Hi-hat, Cowbell, Bas...",None,data/FSD50K/eval_audio/368610.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...
27870,FSD50K,test,431330,background,"(Cello, Brass_instrument, Wind_instrument_and_...",None,data/FSD50K/eval_audio/431330.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...



* Label pairs distribution:


,count,percentage
labels,,
"(Bass_drum, Cowbell, Hi-hat, Snare_drum, Synthesizer, Trombone, Trumpet)",1,50.0
"(Brass_instrument, Cello, Double_bass, Drum, Orchestra, Pizzicato, Wind_instrument_and_woodwind_instrument)",1,50.0



 ---- 


### Examine single vs. multi-label trigger data

In [12]:
trigger_subset = combined_meta[combined_meta["label_type"] == "trigger"]
label_len = trigger_subset["labels"].apply(len)
for label_count in sorted(label_len.unique()):
    subset = trigger_subset[label_len == label_count]
    print_subset_info(f"Number of labels = {label_count}", subset)
    print_value_counts_pcts("\n* Label pairs distribution:", normalize_label_tuple(subset["labels"]))

    print("\n ---- ")

* Total samples: 2389

* Distribution by source dataset:


,count,percentage
source_dataset,,
FSD50K,2139,89.535370
ESC50,200,8.371704
FOAMS,50,2.092926



* Distribution by split:


,count,percentage
split,,
train,1641,68.689828
test,525,21.975722
val,223,9.334450



* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by:


...


* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by: All samples have the same value: None

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,dataset_license,sound_license
29333,FSD50K,test,259513,trigger,"(human_breathing, clearing_throat)",None,data/FSD50K/eval_audio/259513.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
29323,FSD50K,test,425777,trigger,"(human_breathing, clearing_throat)",None,data/FSD50K/eval_audio/425777.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/pu...
29215,FSD50K,test,408095,trigger,"(chewing_gum, human_breathing)",None,data/FSD50K/eval_audio/408095.wav,{'license_url': 'https://creativecommons.org/l...,{'license_url': 'http://creativecommons.org/li...



* Label pairs distribution:


,count,percentage
labels,,
"(clearing_throat, human_breathing)",18,64.285714
"(chewing_gum, human_breathing)",4,14.285714
"(chewing_gum, clearing_throat)",2,7.142857
"(swallowing, water_drops)",2,7.142857
"(chewing_gum, plastic_crumpling)",2,7.142857



 ---- 
